<h1> 1. Multivariate Normal Mixture Model</h1>
This notebook shows how the E(xpectation) M(aximization) algorithm for multivariate normal mixtures is implemented in code. One iteration of EM is simply:

<code>m,μ,Σ = EM_normal(data, m,μ,Σ)</code>

and is fully contained in the cell below. All other code in this notebook amounts to generating some random <code>data</code> to fit the model on, and visualizing the process.

In [1]:
pdf_normal(n,v,μ,Σ) = (2*π)^(-0.5*n) * det(Σ)^(-0.5) * exp(-0.5*(v-μ)'*inv(Σ)*(v-μ))

function EM_normal(t, m,μ,Σ)
    P = size(t,1); N = size(t,2); G = length(Σ)
    
    #E-step
    r = zeros(N,G)
    for g=1:G, n=1:N
        r[n,g] = m[g]*pdf_normal(P,t[:,n],μ[:,g],Σ[g])
    end
    r = r./sum(r,2)
    
    #M-step
    rsum1 = sum(r,1)
    m = (rsum1/N)'
    μ = (t*r)./rsum1
    for g=1:G
        Σ[g] = sum([r[n,g] * (t[:,n]-μ[:,g])*(t[:,n]-μ[:,g])' for n=1:N])./rsum1[g]
    end
    return m, μ, Σ
end

EM_normal (generic function with 1 method)

<h1> 2. Some functions for visualization</h1>
These functions assume 2 dimensional data. All other code is general for P dimensional data.

In [2]:
function T(Σ)
    #make a tranformation matrix from Σ via eigenvectors/values
    V,λ,iV = svd(Σ)
    T = V*diagm(sqrt.(λ))
    return T
end

function visualize(v,m,μ,Σ)
    p = plot()
    scatter!(p, v[1,:],v[2,:], title=string("proportions: ",round.(m,2)), lab="data", color=:black)
    
    α = collect(0:0.1:2.1*π)
    c=[cos.(α) sin.(α)]' #circle points
    for g=1:length(Σ)
        λellipse = T(Σ[g])*c .+ μ[:,g]
        plot!(p, λellipse[1,:],λellipse[2,:], lab=string("g",g), width=2)
    end
    plot(p)
end

function some_2d_data(N)
    n=Int(N/4)
    t1=[1 0.5; 0.5 1]*randn(2,n)*0.4.+[-1,-1]
    t2=[1 -0.5; -0.5 1]*randn(2,n)*0.4.+[-1,+1]
    t3=[1 0.25; 0.25 1]*randn(2,n)*0.4.+[+1,-1]
    t4=[1 -0.25; -0.25 1]*randn(2,n)*0.4.+[+0.5,+0.5]
    t=hcat(t1,t2,t3,t4)
    return t
end

some_2d_data (generic function with 1 method)

<h1> 3. Let EM algorithm optimize parameters of the mixture model

In [3]:
using Plots; gr(xlim=(-2,2), ylim=(-2,2), size=(600,600))

t = some_2d_data(400)
P=size(t,1) #dimensionality of data
G=4 #nr of components to fit

#initial mixture component parameters
m=ones(G)/G
μ=randn(P,G)*0.1
Σ=[eye(P) for g=1:G]

#optimize mixture component parameters
@gif for ITER=1:100
    m,μ,Σ = EM_normal(t, m,μ,Σ)
    visualize(t,m,μ,Σ)
end

INFO: Saved animation to /home/andy/dev/githubnotebooks/MixtureModeling/Multivariate_Gaussians/tmp.gif


Plots.AnimatedGif("/home/andy/dev/githubnotebooks/MixtureModeling/Multivariate_Gaussians/tmp.gif")

// Anders